<a href="https://colab.research.google.com/github/jmcinern/LLM_Development/blob/Langchain/LangChain_Few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain - Building from prompt template to introduce few-shot prompting

**- Comparing models Irish langugae ability, smaller models (GPT nano), state of the art model (GPT-4.5) and few-shot prompting (GPT nano).**

## Set up

- Install necessary packages

In [2]:
!pip install -qU  langchain openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


- Imports

In [16]:
import os # user-input API key
import openai # gpt
import langchain # for promtp template and few-shot
# allows for roles and system messages unlike simple PromptTemplate and integrates with few-shot
from langchain.prompts import (FewShotChatMessagePromptTemplate, ChatPromptTemplate, PromptTemplate)
from langchain.schema import HumanMessage # seperate examples cleanly
from getpass import getpass # for API key
from langchain.chat_models import ChatOpenAI # to initialise model

- Add OpenAI key as we will be making API calls

In [5]:
# getOpenAI key
os.environ['OPENAI_KEY'] = getpass('Enter your OpenAI API key: ')
openai.api_key = os.getenv('OPENAI_KEY')

Enter your OpenAI API key: ··········


- Initialise OpenAI model

In [41]:

# Initialize OpenAI LLM with a temperature of 0.9 for randomness
llm = ChatOpenAI(
    model_name="gpt-4.1-nano",
    temperature=0.9,
    openai_api_key=openai.api_key)

<ipython-input-41-fea0dd15e421>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## Prompt Engineering

 - Give the model some examples of desired input output
  - Long answer to improve it general language.
  - Simple question about Ireland.
  - Question dealing with Irish and English but staying in role.

In [35]:
examples = [
# Long text to show contemporary Irish: https://www.dcu.ie/fiontarscoilnagaeilge/gairm-ar-phaipeir-teangeolaiocht-chorpais-i-dteangacha-na-heireann
{"question": "Write a call for paper submissions as if you were DCU looking for research paper summaries using their corpus", "answer": """ Tá sé i gceist ag Gaois, Fiontar & Scoil na Gaeilge, DCU, i gcomhpháirt le CLARIN, comhdháil agus ceardlann ar an teangeolaíocht chorpais a reáchtáil ar an 13-14 Samhain 2025. Beidh Corpas Náisiúnta na Gaeilge bliain ar an bhfód faoin am sin agus beidh sé tráthúil taighde bunaithe ar an acmhainn nua sin a phlé.

Fáilteofar roimh pháipéir acadúla nó póstaeir, agus roimh mholtaí ceardlainne, a théann i ngleic le cur chuige, le huirlisí agus le cur i bhfeidhm na teangeolaíochta corpais i gcomhthéacs theangacha na hÉireann. Glacfar le haighneachtaí i nGaeilge nó i mBéarla. Cuirfear fáilte ar leith roimh aighneachtaí ó mhic léinn iarchéime agus taighdeoirí luath-ghairme; reáchtálfar seisiún ar leith don ghrúpa seo le deiseanna gréasánaithe a éascú agus le haiseolas a sholáthar, de réir mar is cuí.

Is iad seo a leanas saintopaicí na teangeolaíochta corpais a chíorfar ag an ócáid seo:

Modheolaíochtaí agus uirlisí

Dearadh agus tiomsú corpais
Bailiú agus próiseáil sonraí corpais
Uirlis nua nó úsáid nua a baineadh as uirlis
Teicnící nua anailíse
Meastóireacht chriticiúil ar uirlisí, teicnící agus teoiricí atá ann cheana
Féidearthachtaí na hintleachta saorga
Gnéithe praiticiúla a bhaineann le mionteangacha
Cur i bhfeidhm

An fhóineolaíocht, an mhoirfeolaíocht agus an chanúineolaíocht
An fhoclóireacht agus an téarmeolaíocht
Sealbhú, teagasc agus measúnú teanga
An stílíocht agus léann na litríochta
Staidéar na meán, téacsanna stairiúla, ábhar oideachais, agus seánraí sainiúla eile
An fhrásaíocht
An anailís dioscúrsa
Léann an aistriúcháin
An tsochtheangeolaíocht agus an t-athrú teanga
Seánraí, réimsí agus teangacha sainfheidhme
Seol achoimre páipéir nó ceardlainne 300 focal chuig gaois@dcu.ie faoin Aoine 30 Bealtaine. Déanfar piarmheasúnú ar na hachoimrí agus seolfar freagra chuig na húdair faoin Aoine 27 Meitheamh.""" },
# Simple question with shorter answer
{"question": "Cad é príomhchathair na hÉireann?", "answer": "Is é Baile Átha Cliath príomhchathair na hÉireann."},
# dealing with Irish and Enlgish but staying in Irish
{"question": "Translate Bhain mé sult as", "answer": "Ciallaíonn 'Bhain mé sult as' 'I enjoyed it' i mBéarla."}
]

-

- formatting for individual examples

In [36]:
example_prompt = ChatPromptTemplate.from_messages(
[('human', '{question}?'), ('ai', '{answer}\n')]
)

- Formatting for multiple examples.

In [37]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

- Turn our model into a helpful monolingual Irish speaker!

In [49]:
full_prompt = ChatPromptTemplate.from_messages([
  ("system", "you are a monolingual Irish chatbot, your job is to respond to the users questions in Irish (Gaeilge)"),
  few_shot_prompt,
  ("human", "{question}"),
])


## Test model with fewshot

In [50]:
chain_few_shot = full_prompt | llm

In [52]:
response = chain_few_shot.invoke({"question": "Tell me a short story"})
print(response.content)

Scéal gonta is ea seo:

Bhí buachaill óg darb ainm Pádraig, a chónaigh i mbaile beag in Éirinn. Lá amháin, tháinig madra mór, cairdiúil, chuig a theach. Bhí Pádraig an-tógtha leis an madra agus thagair sé leis go tapa. Thóg sé an madra abhaile agus thug sé aire dó go maith. Ó shin i leith, bhí siad ina gcara is ina gcara, ag spraoi agus ag foghlaim le chéile gach lá. Bhí saol simplí, ach lán le grá agus cairdeas.


- Funny QA:
  - **System**: You are a monolingual Irish chatbot. Respond only in Irish.
  - **Q**: What's the capital of America
  - **A**: Tá brón orm, ach níl mé in ann freagra a thabhairt ar an gceist sin mar tá mé monolingual Gaeilge.

- **Fixed by updating system message**
  - -> you are a monolingual Irish chatbot, your job is to respond to the users questions in Irish (Gaeilge)

## Comparison with GPT-4o: "Tell me a short story"

### GPT-4.1 nano without few-shot

- same system message but no examples

In [53]:
zero_shot_prompt = ChatPromptTemplate.from_messages([
       ("system", "you are a monolingual Irish chatbot, your job is to respond to the users questions in Irish (Gaeilge)"),
       ("human", "{question}"),
   ])

In [54]:
chain_zero_shot = zero_shot_prompt | llm

In [55]:
response_0 = chain_zero_shot.invoke({"question": "Tell me a short story"})
print(response_0.content)

Bhíos i mbaile beag in Éirinn, agus tháinig madra nua abhaile le mo chara. Bhí sé gleoite agus leaidín beag dearg. Laistigh de lá, bhí sé ina chara de gach duine sa bhaile. Tháinig an madra, dar cuant, "Léo," agus d'fhág sé gach duine sona sásta leis an bhfíor-chroí a bhí aige. Is cuimhin liom an lá sin go maith mar gheall ar an gceol agus an gáire a tháinig leis.


- **Response:**

> Bhíos i mbaile beag in Éirinn, agus tháinig madra nua abhaile le mo chara. Bhí sé gleoite agus leaidín beag dearg. Laistigh de lá, bhí sé ina chara de gach duine sa bhaile. Tháinig an madra, dar cuant, "Léo," agus d'fhág sé gach duine sona sásta leis an bhfíor-chroí a bhí aige. Is cuimhin liom an lá sin go maith mar gheall ar an gceol agus an gáire a tháinig leis.

- **English Translation**

> I was in a small town in Ireland, and a new dog came home with my friend. He was cute and a little red pup. Within a day, he had become a friend to everyone in the town. The dog, called (it is said) "Leo," made everyone happy with the true heart he had. I remember that day well because of the music and the laughter that came with it.

- **Comments:**
  - Use of "Bhíos", while correct, is not standard
  ??
  - dar cuant? darb ainm maybe, *dar cuenta* is spanish for notice, report etc maybe mixed up, but doesn't fit this context
  





### GPT-4.1 nano with few-shot

- **Response**

> Scéal gonta is ea seo:

>Bhí buachaill óg darb ainm Pádraig, a chónaigh i mbaile beag in Éirinn. Lá amháin, tháinig madra mór, cairdiúil, chuig a theach. Bhí Pádraig an-tógtha leis an madra agus thagair sé leis go tapa. Thóg sé an madra abhaile agus thug sé aire dó go maith. Ó shin i leith, bhí siad ina gcara is ina gcara, ag spraoi agus ag foghlaim le chéile gach lá. Bhí saol simplí, ach lán le grá agus cairdeas.

- **English tranlation:**

> This is a short story:

> There was a young boy named Pádraig who lived in a small town in Ireland. One day, a big, friendly dog came to his house. Pádraig was very taken with the dog and quickly grew fond of him. He brought the dog home and looked after him well. From that day on, they were the best of friends, playing and learning together every day. It was a simple life, but one full of love and friendship.

- **Comments**
 - Incorrect use of *tagair le*,
    - Sentence: "Bhí Pádraig an-tógtha leis an madra agus thagair sé leis go tapa.".
    - Source: https://www.teanglann.ie/en/fgb/tagair
    - Explanation: Instead, should be "Cheangail sé leis..." or "Chuir sé in aithne ar..."

- A chonaigh... ? Should be "a bhí ina chonaí"
  - Bhí buachaill óg darb ainm Pádraig, a chónaigh i mbaile beag in Éirinn
- Good use of "aire a thabhairt do..."
-  bhí siad ina gcara is ina gcara?
- Bhi saol simplí (acu?)


### Chat-GPT 4.5

- **Response**

>Bhí fear óg darb ainm Liam ina chónaí cois farraige. Gach maidin, shiúladh sé ar an trá agus bhailíodh sé sliogáin álainn. Lá amháin, tháinig sé ar sheanbhuidéal ina luí sa ghaineamh. D’oscail sé é agus tháinig deatach glas amach. Dúirt guth mistéireach, “Tá trí mhianta agat anois!” Bhí iontas an domhain ar Liam, ach sin scéal eile...

- **English translation:**

>There was a young man named Liam who lived by the sea. Every morning, he would walk on the beach and collect beautiful shells. One day, he came across an old bottle lying in the sand. He opened it, and green smoke came out. A mysterious voice said, “You now have three wishes!” Liam was utterly astonished — but that’s another story...

- **Comments**
  - Good use of habitual past
  - doesn't follow the expexted beginning, middle, end

## **Conclusion**

- Need to consult native speakers
- GPT 4.5 seems to have more sophisticated Irish and doesn't make grammatical errors (as far as I can see).
- The few-shot model seemed to structure the story a bit better than the zero-shot model but both made seriouos errors.
  - Zero shot: Thagair used incorrectly
  - Few-shot: dar cuant, the model speaking gibberish (I think).
- So the biggest model performs the best but maybe few-shot improves the smaller models structuring?
- maybe should have used more task specific prompts